## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [42]:
#!pip install -U tensorflow --quiet

     |████████████████████████████████| 86.3MB 95kB/s 
     |████████████████████████████████| 3.8MB 31.8MB/s 
     |████████████████████████████████| 450kB 57.4MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.


In [1]:
import tensorflow as tf
#tf.compat.v1.enable_eager_execution ()
import numpy as np

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/R6/prices.csv')

### Check all columns in the dataset

In [5]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],inplace=True)

In [7]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [8]:
data['volume'] = data['volume']/1000000
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [10]:
X=data.drop('volume',axis=1)
y=data['volume']
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

Using TensorFlow backend.


#### Convert Training and Test Data to numpy float32 arrays


In [11]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

X_norm = normalizer.fit_transform(X)

X_norm = X_norm.astype('float32')
X_train, X_test, y_train, y_test = train_test_split(X_norm, encoded_Y, test_size=0.4, random_state=42)
X_train.shape

(510758, 4)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
#Used above and changed to float 32

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:

w =tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [17]:

tf.executing_eagerly()

False

In [19]:
for i in range(90):
    
    w, b = train(X_train, encoded_Y, w, b, learning_rate=0.01)
    
    print('Current Loss on iteration', i, 
          loss(y_train, prediction(X_train, w, b)))

Current Loss on iteration 0 Tensor("Mean_21:0", shape=(), dtype=float32)
Current Loss on iteration 1 Tensor("Mean_23:0", shape=(), dtype=float32)
Current Loss on iteration 2 Tensor("Mean_25:0", shape=(), dtype=float32)
Current Loss on iteration 3 Tensor("Mean_27:0", shape=(), dtype=float32)
Current Loss on iteration 4 Tensor("Mean_29:0", shape=(), dtype=float32)
Current Loss on iteration 5 Tensor("Mean_31:0", shape=(), dtype=float32)
Current Loss on iteration 6 Tensor("Mean_33:0", shape=(), dtype=float32)
Current Loss on iteration 7 Tensor("Mean_35:0", shape=(), dtype=float32)
Current Loss on iteration 8 Tensor("Mean_37:0", shape=(), dtype=float32)
Current Loss on iteration 9 Tensor("Mean_39:0", shape=(), dtype=float32)
Current Loss on iteration 10 Tensor("Mean_41:0", shape=(), dtype=float32)
Current Loss on iteration 11 Tensor("Mean_43:0", shape=(), dtype=float32)
Current Loss on iteration 12 Tensor("Mean_45:0", shape=(), dtype=float32)
Current Loss on iteration 13 Tensor("Mean_47:0",

### Get the shapes and values of W and b

In [20]:
print('Weights:\n', w)
print('Bias:\n',b)

Weights:
 Tensor("sub_397:0", shape=(4, 1), dtype=float32)
Bias:
 Tensor("sub_398:0", shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [22]:
predict = prediction(X_test,w,b)
predict

<tf.Tensor 'Add_200:0' shape=(340506, 1) dtype=float32>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [91]:
data_iris = pd.read_csv('/gdrive/My Drive/R6/Iris.csv')
data_iris.drop('Id',axis=1,inplace=True)
data_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
X_iris=data_iris.drop('Species',axis=1)
y_iris=data_iris['Species']
encoded_y = encoder.fit_transform(y_iris)
X_iris_norm = normalizer.fit_transform(X_iris)

### Splitting the data into feature set and target set

In [93]:
X_iris_norm = X_iris_norm.astype('float32')
X_train, X_test, y_train, y_test = train_test_split(X_iris_norm, encoded_y, test_size=0.30, random_state=42)
X_train.shape

(105, 4)

In [0]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
#model.add(tf.keras.layers.Reshape(input_shape=(4,)))

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3,input_shape=(4,) ,activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [103]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=300,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/300
105/105 [==============================] - 0s 143us/sample - loss: 0.9834 - acc: 0.4667 - val_loss: 0.9732 - val_acc: 0.5778
Epoch 2/300
105/105 [==============================] - 0s 106us/sample - loss: 0.9830 - acc: 0.4667 - val_loss: 0.9729 - val_acc: 0.5778
Epoch 3/300
105/105 [==============================] - 0s 135us/sample - loss: 0.9827 - acc: 0.4667 - val_loss: 0.9722 - val_acc: 0.5778
Epoch 4/300
105/105 [==============================] - 0s 101us/sample - loss: 0.9823 - acc: 0.4952 - val_loss: 0.9715 - val_acc: 0.6000
Epoch 5/300
105/105 [==============================] - 0s 96us/sample - loss: 0.9819 - acc: 0.4667 - val_loss: 0.9706 - val_acc: 0.6000
Epoch 6/300
105/105 [==============================] - 0s 93us/sample - loss: 0.9815 - acc: 0.4762 - val_loss: 0.9703 - val_acc: 0.6222
Epoch 7/300
105/105 [==============================] - 0s 96us/sample - loss: 0.9811 - acc: 0.5048 - val_loss: 0.9703 - val_acc: 0.6000

### Model Prediction

In [104]:
prediction = model.predict(X_test[0:1])
import numpy as np
predicted_num = np.argmax(prediction)
predicted_num

1

### Save the Model

In [0]:
model.save('iris_model.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?